# Robustness Experiment Evaluation

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [7]:
file = "n1_RobustnessExperiment_2022-01-17_11-23-01.csv"


In [8]:
df_frame = pd.read_csv(file)
df_frame['makespan_std'] = df_frame['makespan'] - df_frame['makespan-optimum']
df_frame['makespan_uncertainty_std'] = df_frame['makespan_uncertainty'] - df_frame['makespan-optimum']
df_frame['makespan_uncertainty_baseline_delay'] = df_frame['makespan_uncertainty_std'] - df_frame['makespan_std']
list_ordering = {"RandomSolver": 0, "HillClimbing": 1, "TabuSearch": 2, "SimulatedAnnealing": 3, "GeneticAlgorithm": 4}
df_frame["solver"] = pd.Categorical(df_frame["solver"], categories=list_ordering)
df_frame.loc[~pd.notna(df_frame['robustness_measure']), 'robustness_measure'] = "No-RM"
df_frame.head(100)

,benchmark,solver,iterations,makespan-optimum,makespan,robustness,robustness_measure,uncertainty_percentage,makespan_uncertainty,robustness_uncertainty,makespan_std,makespan_uncertainty_std,makespan_uncertainty_baseline_delay
0,n1.mm/n110_1.mm,SimulatedAnnealing,5000,17,17,7.2,SF3_W1,0.0,17,7.2,0,0,0
1,n1.mm/n110_1.mm,SimulatedAnnealing,5000,17,17,7.2,SF3_W1,0.0,17,7.2,0,0,0
2,n1.mm/n110_1.mm,SimulatedAnnealing,5000,17,17,6.0,SF3_W1,0.0,17,6.0,0,0,0
3,n1.mm/n110_1.mm,SimulatedAnnealing,5000,17,17,7.2,SF3_W1,0.0,17,7.2,0,0,0
4,n1.mm/n110_1.mm,SimulatedAnnealing,5000,17,17,6.8,SF3_W1,0.0,17,6.8,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,n1.mm/n110_1.mm,SimulatedAnnealing,5000,17,17,6.8,SF3_W1,0.0,17,6.8,0,0,0
96,n1.mm/n110_1.mm,SimulatedAnnealing,5000,17,18,7.4,SF3_W1,0.0,18,7.4,1,1,0
97,n1.mm/n110_1.mm,SimulatedAnnealing,5000,17,17,6.8,SF3_W1,0.0,17,6.8,0,0,0
98,n1.mm/n110_1.mm,SimulatedAnnealing,5000,17,17,6.8,SF3_W1,0.0,17,6.8,0,0,0


In [9]:
df_grouped = df_frame.groupby(['uncertainty_percentage', 'iterations', 'robustness_measure'])['makespan_uncertainty_baseline_delay'].describe()[['mean', 'std']]
df_grouped.head()

mean  std
uncertainty_percentage iterations robustness_measure           
0.0                    500        No-RM                0.0  0.0
                                  SF1                  0.0  0.0
                                  SF1_W1               0.0  0.0
                                  SF1_W9               0.0  0.0
                                  SF2                  0.0  0.0

In [10]:
# print(proactive_describe.to_latex(float_format="{:0.2f}".format))
#for solver in df_frame['solver'].unique():
group = df_grouped.reset_index()
 #   group = group[group['solver'] == solver]
group = group.set_index(['iterations', 'robustness_measure'])
group = group.pivot(columns='uncertainty_percentage')
print(group.to_latex(float_format="{:0.2f}".format))

\begin{tabular}{llrrrrrrrr}
\toprule
     & {} & \multicolumn{4}{l}{mean} & \multicolumn{4}{l}{std} \\
     & uncertainty\_percentage & 0.00 & 0.05 & 0.10 & 0.20 & 0.00 & 0.05 & 0.10 & 0.20 \\
iterations & robustness\_measure &      &      &      &      &      &      &      &      \\
\midrule
500  & No-RM & 0.00 & 0.93 & 1.74 & 3.28 & 0.00 & 1.25 & 1.63 & 2.12 \\
     & SF1 & 0.00 & 0.86 & 1.65 & 3.09 & 0.00 & 1.15 & 1.49 & 1.95 \\
     & SF1\_W1 & 0.00 & 0.86 & 1.66 & 3.14 & 0.00 & 1.21 & 1.56 & 2.04 \\
     & SF1\_W9 & 0.00 & 0.90 & 1.69 & 3.18 & 0.00 & 1.26 & 1.64 & 2.15 \\
     & SF2 & 0.00 & 0.82 & 1.61 & 3.09 & 0.00 & 1.16 & 1.58 & 2.06 \\
     & SF2\_W1 & 0.00 & 0.84 & 1.64 & 3.10 & 0.00 & 1.23 & 1.60 & 2.08 \\
     & SF2\_W9 & 0.00 & 0.86 & 1.66 & 3.14 & 0.00 & 1.29 & 1.69 & 2.17 \\
     & SF3 & 0.00 & 0.82 & 1.60 & 3.07 & 0.00 & 1.09 & 1.44 & 1.90 \\
     & SF3\_W1 & 0.00 & 0.85 & 1.66 & 3.14 & 0.00 & 1.17 & 1.56 & 2.07 \\
     & SF3\_W9 & 0.00 & 0.83 & 1.64 & 3.13 & 0.00 & 1.